# 项目：整理Netflix电影演员评分数据

## 分析目标

此数据分析的目的是，整理不同流派影视作品，比如喜剧片、动作片、科幻片中，各演员出演作品的平均IMDB评分，从而挖掘出各个流派中的高评分作品演员。

本实战项目的目的在于练习整理数据，从而得到可供下一步分析的数据。

## 简介

原始数据集记录了截止至2022年7月美国地区可观看的所有Netflix电视剧及电影数据。数据集包含两个数据表：`titles.csv`和`credits.csv`。

`titles.csv`包含电影及电视剧相关信息，包括影视作品ID、标题、类型、描述、流派、IMDB（一个国外的在线评分网站）评分，等等。`credits.csv`包含超过7万名出现在Netflix影视作品的导演及演员信息，包括名字、影视作品ID、人物名、演职员类型（导演/演员）等。

`titles.csv`每列的含义如下：
- id：影视作品ID。
- title：影视作品标题。
- show_type：作品类型，电视节目或电影。
- description：简短描述。
- release_year：发布年份。
- age_certification：适龄认证。
- runtime：每集电视剧或电影的长度。
- genres：流派类型列表。
- production_countries：出品国家列表。
- seasons：如果是电视剧，则是季数。
- imdb_id：IMDB的ID。
- imdb_score：IMDB的评分。
- imdb_votes：IMDB的投票数。
- tmdb_popularity：TMDB的流行度。
- tmdb_score：TMDB的评分。

`credits.csv`每列的含义如下：
- person_ID：演职员ID。
- id：参与的影视作品ID。
- name：姓名。
- character_name：角色姓名。
- role：演职员类型，演员或导演。

## 1、读取数据

In [5]:
import pandas as pd

In [6]:
original_titles = pd.read_csv("titles.csv")
original_credits = pd.read_csv("credits.csv")

In [7]:
original_titles.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600


In [8]:
original_credits.head()

,person_id,id,name,character,role
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR


## 2、评估和清理数据

结构和内容，即整齐度和干净度
数据的结构性问题指不符合“每个变量为一列，每个观察值为一行，每种类型的观察单位为一个表格”这三个标准；数据的内容性问题包括存在丢失数据、重复数据、不一致数据、无效数据等。

In [9]:
cleaned_titles = original_titles.copy()
cleaned_credits = original_credits.copy()

### 2.1 整齐度--结构

#### 1、整理cleaned_titles

根据前面初步提取到的cleaned_titles里的前5行，可知`genres`和`production_countries`的变量中包含多个值，应当进行拆分。
先提取任意一个包含多个值的`genres`变量，用于观察其数据类型。

In [10]:
cleaned_titles['genres'][1]

"['drama', 'crime']"

In [11]:
cleaned_titles['genres'] = cleaned_titles['genres'].apply(lambda s:eval(s))
cleaned_titles['genres'][1]

['drama', 'crime']

In [12]:
cleaned_titles = cleaned_titles.explode("genres")
cleaned_titles.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,documentation,['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,crime,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,drama,['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,action,['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300


针对`production_countries`列进行一样的流程

In [13]:
cleaned_titles['production_countries'][4]

4    ['GB', 'US']
4    ['GB', 'US']
Name: production_countries, dtype: object

In [14]:
#cleaned_titles['production_countries'] = cleaned_titles['production_countries'].apply(lambda s:eval(s))
cleaned_titles['production_countries'][4]

4    ['GB', 'US']
4    ['GB', 'US']
Name: production_countries, dtype: object

In [15]:
cleaned_titles = cleaned_titles.explode('production_countries')
cleaned_titles.loc[4]

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,war,"['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.6
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,action,"['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.6


#### 2、整理cleaned_credits

In [16]:
cleaned_credits.sample(10)

,person_id,id,name,character,role
48646,238170,tm469991,Enzo Casertano,NaN,ACTOR
18071,208441,tm156811,Yogendra Tiku,Rani's Father,ACTOR
62709,1131219,tm844236,Nicolas Aqui,Zee (voice),ACTOR
38563,367004,ts82296,Im Soo-hyang,Kang Mi-Rae,ACTOR
27797,234955,tm244259,David Cardona,BIA Officer (uncredited),ACTOR
4531,3824,tm91952,Amanda Seyfried,Karen Smith,ACTOR
53760,1800292,tm454824,Maxim Khanzhov,NaN,ACTOR
5829,78083,tm133374,Jae Head,S.J. Tuohy,ACTOR
63622,736704,tm454338,Mary Joy Apostol,NaN,ACTOR
70009,2150255,tm1096775,Mike Foreman,Self,ACTOR


根据上面随机提取的10行观察值来看，不存在结构性问题

### 2.2 数据干净度--内容

包括：丢失数据、重复数据、不一致数据、无效数据

1、大致了解：

In [17]:
cleaned_titles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15147 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    15147 non-null  object 
 1   title                 15146 non-null  object 
 2   type                  15147 non-null  object 
 3   description           15125 non-null  object 
 4   release_year          15147 non-null  int64  
 5   age_certification     9298 non-null   object 
 6   runtime               15147 non-null  int64  
 7   genres                15088 non-null  object 
 8   production_countries  15147 non-null  object 
 9   seasons               5923 non-null   float64
 10  imdb_id               14525 non-null  object 
 11  imdb_score            14399 non-null  float64
 12  imdb_votes            14375 non-null  float64
 13  tmdb_popularity       14995 non-null  float64
 14  tmdb_score            14614 non-null  float64
dtypes: float64(5), int64(2), 

存在缺失值的变量：`title`、`description`、`age_certification`、`genres`、`production_countries`、`seasons`、`imdb_id`、`imdb_score`、`tmdb_popularity`、`tmdb_score`、`imdb_votes`、`tmdb_popularity`、`tmdb_score`

2、`release_year`表示年份，数据类型不应为数字，应为日期，所以需要进行数据格式转换。

In [18]:
cleaned_titles['release_year'] = pd.to_datetime(cleaned_titles['release_year'],format='%Y')
cleaned_titles['release_year']

0      1945-01-01
1      1976-01-01
1      1976-01-01
2      1972-01-01
2      1972-01-01
          ...    
5847   2021-01-01
5848   2021-01-01
5849   2021-01-01
5849   2021-01-01
5849   2021-01-01
Name: release_year, Length: 15147, dtype: datetime64[ns]

In [19]:
cleaned_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77801 entries, 0 to 77800
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   person_id  77801 non-null  int64 
 1   id         77801 non-null  object
 2   name       77801 non-null  object
 3   character  68029 non-null  object
 4   role       77801 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.0+ MB


存在缺失值的变量：`character`
此外，`person_id`表示演职员ID，数据类型不应为数字，应为字符串，所以需要进行数据格式转换。

In [20]:
cleaned_credits['person_id'] = cleaned_credits['person_id'].astype('str')
cleaned_credits['person_id']

0           3748
1          14658
2           7064
3           3739
4          48933
          ...   
77796     736339
77797     399499
77798     373198
77799     378132
77800    1950416
Name: person_id, Length: 77801, dtype: object

3、处理缺失数据

cleaned_titles中影响后续分析的、具有缺失值的变量为`imdb_score`和`genres`，即IMDB评分和流派，需要删除这些观察值

In [21]:
cleaned_titles.query("imdb_score.isnull()")

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945-01-01,TV-MA,51,documentation,['US'],1.0,NaN,NaN,NaN,0.600,NaN
75,tm132164,Bill Hicks: Sane Man,MOVIE,Sane Man was filmed before Bill recorded ‘Dang...,1989-01-01,R,80,comedy,['US'],NaN,NaN,NaN,NaN,3.377,7.5
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1991-01-01,TV-G,18,documentation,['JP'],12.0,NaN,NaN,NaN,7.730,7.8
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1991-01-01,TV-G,18,family,['JP'],12.0,NaN,NaN,NaN,7.730,7.8
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1991-01-01,TV-G,18,reality,['JP'],12.0,NaN,NaN,NaN,7.730,7.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5810,tm1225897,Social Man,MOVIE,Two competitive social media Influencers go he...,2021-01-01,NaN,96,drama,[],NaN,tt20198164,NaN,NaN,NaN,NaN
5833,ts307884,HQ Barbers,SHOW,When a family run barber shop in the heart of ...,2021-01-01,TV-14,24,comedy,['NG'],1.0,NaN,NaN,NaN,0.840,NaN
5840,tm1216735,Sun of the Soil,MOVIE,"In 14th-century Mali, an ambitious young royal...",2022-01-01,NaN,26,NaN,[],NaN,NaN,NaN,NaN,1.179,7.0
5844,tm1074617,Bling Empire - The Afterparty,MOVIE,"The stars of ""Bling Empire"" discuss the show's...",2021-01-01,NaN,35,NaN,['US'],NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
cleaned_titles = cleaned_titles.dropna(subset = ['imdb_score'])
cleaned_titles['imdb_score'].isnull().sum()

np.int64(0)

In [23]:
cleaned_titles.query('genres.isnull()')

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
1813,ts77824,My Next Guest Needs No Introduction With David...,SHOW,TV legend David Letterman teams up with fascin...,2018-01-01,TV-MA,50,NaN,['US'],4.0,tt7829834,7.8,5581.0,8.217,7.6
1939,ts215037,Minecraft: Story Mode,SHOW,"MInecraft: Story Mode is an interactive, anima...",2018-01-01,TV-PG,52,NaN,['US'],1.0,tt10498322,5.6,347.0,NaN,NaN
2386,ts74805,A Little Help with Carol Burnett,SHOW,In this unscripted series starring comedy lege...,2018-01-01,TV-G,24,NaN,['US'],1.0,tt7204366,6.3,237.0,1.621,6.2
2658,ts265844,#ABtalks,SHOW,#ABtalks is a YouTube interview show hosted by...,2018-01-01,TV-PG,68,NaN,[],1.0,tt12635254,9.6,7.0,NaN,NaN
4274,tm1172010,The Lockdown Plan,MOVIE,NaN,2020-01-01,NaN,49,NaN,[],NaN,tt13079112,6.5,NaN,NaN,NaN
4648,tm1113921,In Vitro,MOVIE,'In Vitro' is an otherworldly rumination on me...,2019-01-01,NaN,27,NaN,[],NaN,tt10545994,7.7,NaN,NaN,NaN


In [24]:
cleaned_titles = cleaned_titles.dropna(subset=['genres'])
cleaned_titles['genres'].isnull().sum()

np.int64(0)

In [25]:
cleaned_titles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14393 entries, 1 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id                    14393 non-null  object        
 1   title                 14393 non-null  object        
 2   type                  14393 non-null  object        
 3   description           14388 non-null  object        
 4   release_year          14393 non-null  datetime64[ns]
 5   age_certification     8958 non-null   object        
 6   runtime               14393 non-null  int64         
 7   genres                14393 non-null  object        
 8   production_countries  14393 non-null  object        
 9   seasons               5655 non-null   float64       
 10  imdb_id               14393 non-null  object        
 11  imdb_score            14393 non-null  float64       
 12  imdb_votes            14371 non-null  float64       
 13  tmdb_popularity       

cleaned_credits中的缺失变量`character`并不影响本次的分析，保留即可

4、处理重复数据

`cleaned_titles`和`cleaned_credits`中不应该存在每个变量值都相同的观察值

In [26]:
cleaned_titles.duplicated().sum()

np.int64(0)

In [27]:
cleaned_credits.duplicated().sum()

np.int64(0)

输出结果为0，说明不存在重复值。


5、处理不一致数据

cleaned_titles中不一致数据可能存在于`genres`和`production_countries`变量中
PS：若仅仅针对此次分析目标，`production_countries`变量值不一致其实不影响分析，但是为了数据的清洁度，我们还是一起处理掉

In [28]:
cleaned_titles['genres'].value_counts()

genres
drama            2827
comedy           2218
thriller         1180
action           1109
romance           955
crime             909
documentation     859
family            652
animation         630
fantasy           619
scifi             560
european          428
horror            366
history           254
music             244
reality           219
sport             170
war               155
western            39
Name: count, dtype: int64

In [29]:
cleaned_titles.query('genres == ""')

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score


不存在多个不同值指代同一流派，且不存在`genres`为空字符串的情况

In [30]:
with pd.option_context('display.max_rows',None):
    print(cleaned_titles['production_countries'].value_counts())

production_countries
['US']                                        4587
['IN']                                        1545
['JP']                                         985
['KR']                                         618
['GB']                                         554
['ES']                                         466
['FR']                                         348
[]                                             300
['CA']                                         255
['MX']                                         198
['TR']                                         192
['BR']                                         186
['PH']                                         180
['DE']                                         179
['CN']                                         164
['AU']                                         150
['ID']                                         143
['IT']                                         139
['CA', 'US']                                   127
['TW']    

`LB`和`Lebanon`都在表示同一国家，需要进行统一。
其余出品国家都用两位的国家代码来表示

In [31]:
cleaned_titles['production_countries'] = cleaned_titles['production_countries'].replace({"Lebanon":"LB"})
# 检查"Lebanon"是否还存在
cleaned_titles.query('production_countries == "Lebanon"')

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score


`cleaned_credits`中不一致数据可能存在于`role`变量中，查看是否存在多个不同值指代同一演职员类型的情况。

In [32]:
cleaned_credits['role'].value_counts()

role
ACTOR       73251
DIRECTOR     4550
Name: count, dtype: int64

从以上输出结果来看，`role`只有两种可能的值，`ACTOR`或`DIRECTOR`，不存在不一致数据。我们可以把这列的类型转换为`Category`，好处是比字符串类型更节约内存空间，也能表明说值的类型有限。

In [33]:
cleaned_credits['role'] = cleaned_credits['role'].astype("category")
cleaned_credits['role']

0           ACTOR
1           ACTOR
2           ACTOR
3           ACTOR
4           ACTOR
           ...   
77796       ACTOR
77797       ACTOR
77798       ACTOR
77799       ACTOR
77800    DIRECTOR
Name: role, Length: 77801, dtype: category
Categories (2, object): ['ACTOR', 'DIRECTOR']

6、处理无效或错误数据

In [34]:
cleaned_titles.describe()

,release_year,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
count,14393,14393.000000,5655.000000,14393.000000,1.437100e+04,14268.000000,13975.000000
mean,2015-12-30 06:28:41.336760832,78.326270,2.424934,6.534802,2.836162e+04,30.008057,6.885143
min,1954-01-01 00:00:00,0.000000,1.000000,1.500000,5.000000e+00,0.600000,1.000000
25%,2015-01-01 00:00:00,44.000000,1.000000,5.800000,7.080000e+02,3.911750,6.200000
50%,2018-01-01 00:00:00,87.000000,1.000000,6.600000,3.166000e+03,9.666000,6.985000
75%,2020-01-01 00:00:00,106.000000,3.000000,7.400000,1.432100e+04,22.766000,7.600000
max,2022-01-01 00:00:00,225.000000,42.000000,9.500000,2.294231e+06,2274.044000,10.000000
std,NaN,40.555015,2.867856,1.157513,1.041849e+05,99.359539,1.112448


从以上统计信息来看，`cleaned_titles`里不存在脱离现实意义的数值。
`cleaned_credits`由于不包含表示数值含义的变量，因此无需用`describe`检查。

## 3、整理数据

In [35]:
cleaned_titles

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,crime,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972-01-01,R,109,drama,['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972-01-01,R,109,action,['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972-01-01,R,109,thriller,['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5846,tm898842,C/O Kaadhal,MOVIE,A heart warming film that explores the concept...,2021-01-01,NaN,134,drama,[],NaN,tt11803618,7.7,348.0,NaN,NaN
5847,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,2021-01-01,NaN,90,comedy,['CO'],NaN,tt14585902,3.8,68.0,26.005,6.300
5849,ts271048,Mighty Little Bheem: Kite Festival,SHOW,"With winter behind them, Bheem and his townspe...",2021-01-01,NaN,7,family,[],1.0,tt13711094,7.8,18.0,2.289,10.000
5849,ts271048,Mighty Little Bheem: Kite Festival,SHOW,"With winter behind them, Bheem and his townspe...",2021-01-01,NaN,7,animation,[],1.0,tt13711094,7.8,18.0,2.289,10.000


In [36]:
cleaned_credits

,person_id,id,name,character,role
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR
...,...,...,...,...,...
77796,736339,tm1059008,Adelaida Buscato,María Paz,ACTOR
77797,399499,tm1059008,Luz Stella Luengas,Karen Bayona,ACTOR
77798,373198,tm1059008,Inés Prieto,Fanny,ACTOR
77799,378132,tm1059008,Isabel Gaona,Cacica,ACTOR


1、根据需要连接两个表，获得综合信息

In [37]:
titles_with_credits = pd.merge(cleaned_credits,cleaned_titles,on="id",how="inner")

In [38]:
titles_with_credits.head()

,person_id,id,name,character,role,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
1,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,crime,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
3,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,crime,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
4,7064,tm84618,Albert Brooks,Tom,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179


2、去除掉不需要的DIRECTOR信息

In [39]:
actor_with_titles = titles_with_credits.query('role == "ACTOR"')

3、先分组

In [40]:
groupby_genres_and_person_id = actor_with_titles.groupby(["genres","person_id"])

4、再聚合

In [41]:
imdb_sorted = groupby_genres_and_person_id["imdb_score"].mean()
with pd.option_context('display.max_rows', 100):   
    print(imdb_sorted)

genres   person_id
action   1000         6.866667
         100007       7.000000
         100013       6.400000
         100019       6.500000
         100020       6.500000
                        ...   
western  993735       6.500000
         998673       7.300000
         998674       7.300000
         998675       7.300000
         99940        4.000000
Name: imdb_score, Length: 168881, dtype: float64


5、上面得到的是具有层次化索引的series，可以使用`reset_index`将其重置为dataframe

In [42]:
imdb_score_sorted_df = imdb_sorted.reset_index()

In [43]:
imdb_score_sorted_df

,genres,person_id,imdb_score
0,action,1000,6.866667
1,action,100007,7.000000
2,action,100013,6.400000
3,action,100019,6.500000
4,action,100020,6.500000
...,...,...,...
168876,western,993735,6.500000
168877,western,998673,7.300000
168878,western,998674,7.300000
168879,western,998675,7.300000


6、接下来找出各个流派的作品中，最高的演员评分及对应的演员名字

In [44]:
genres_max_scores = imdb_score_sorted_df.groupby("genres")["imdb_score"].max()
#imdb_score_sorted_df.groupby("genres")[["imdb_score","person_id"]].max()
# 我们要找的不是每个流派中person_id的最大值，而是每个流派中评分的最大值，以及这个最大值对应的person_id
genres_max_scores 

genres
action           9.3
animation        9.3
comedy           9.2
crime            9.5
documentation    9.1
drama            9.5
european         8.9
family           9.3
fantasy          9.3
history          9.1
horror           9.0
music            8.8
reality          8.9
romance          9.2
scifi            9.3
sport            9.1
thriller         9.5
war              8.8
western          8.9
Name: imdb_score, dtype: float64

In [45]:
genres_max_scores_with_person_id = pd.merge(imdb_score_sorted_df,genres_max_scores,on = ["genres","imdb_score"],how = "inner")
genres_max_scores_with_person_id

,genres,person_id,imdb_score
0,action,12790,9.3
1,action,1303,9.3
2,action,21033,9.3
3,action,336830,9.3
4,action,86591,9.3
...,...,...,...
131,war,826547,8.8
132,western,22311,8.9
133,western,28166,8.9
134,western,28180,8.9


7、再继续找出person_id对应的演员名字，需要将df`genres_max_scores_with_person_id`与cleaned_credits进行merge,由于cleaned_credits还包含其他列(不需要分析的属性)，我们在merge之前需要先将所需要的两列提取出来(name和person_id)；然后因为同一演员可能会出演多部作品，所以还需把重复值删除

In [46]:
actor_id_with_name = cleaned_credits[["person_id","name"]].drop_duplicates()

In [47]:
actor_id_with_name.head()

,person_id,name
0,3748,Robert De Niro
1,14658,Jodie Foster
2,7064,Albert Brooks
3,3739,Harvey Keitel
4,48933,Cybill Shepherd


8、进行连接

In [48]:
genres_max_scores_with_id_name = pd.merge(genres_max_scores_with_person_id,actor_id_with_name,on = "person_id",how="inner")
genres_max_scores_with_id_name

,genres,person_id,imdb_score,name
0,action,12790,9.3,Olivia Hack
1,action,1303,9.3,Jessie Flower
2,action,21033,9.3,Zach Tyler
3,action,336830,9.3,André Sogliuzzo
4,action,86591,9.3,Cricket Leigh
...,...,...,...,...
131,war,826547,8.8,Yuto Uemura
132,western,22311,8.9,Koichi Yamadera
133,western,28166,8.9,Megumi Hayashibara
134,western,28180,8.9,Unsho Ishizuka


In [ ]:
#genres_max_scores_with_id_name = genres_max_scores_with_id_name.sort_values("genres").reset_index().drop("index",axis=1)

In [51]:
genres_max_scores_with_id_name

,genres,person_id,imdb_score,name
0,action,12790,9.3,Olivia Hack
1,action,1303,9.3,Jessie Flower
2,action,21033,9.3,Zach Tyler
3,action,336830,9.3,André Sogliuzzo
4,action,86591,9.3,Cricket Leigh
...,...,...,...,...
131,war,826547,8.8,Yuto Uemura
132,western,28180,8.9,Unsho Ishizuka
133,western,22311,8.9,Koichi Yamadera
134,western,28166,8.9,Megumi Hayashibara
